In [76]:
# 1623 티스토리
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

menus = pd.read_csv('./source.csv')
menus

,업소명,소재지,대분류,메뉴,별점,별점 수
0,이래도식당,대전광역시 동구 백룡로57번길 73,중식,"짜장면, 짬뽕, 우동, 라면, 냉면, 울면, 만두국, 콩국수, 볶음밥, 김치볶음밥, 짬뽕밥, 잡채밥, 오므라이스 오징어덮밥, 육개장, 뚝배기불고기, 갈비탕",4.17,52
1,이삭토스트,대전광역시 동구 동대전로131번길 8-5,샌드위치,"햄토스트, 불고기토스트, 치킨토스트, 피자토스트, 포테이토토스트, 감자튀김",4.56,272
2,고구려명장식당,대전광역시 동구 백룡로57번길 184,한식,"삼계탕, 닭볶음탕",4.42,225
3,미가 칼국수,대전광역시 동구 백룡로 3,칼국수,"칼국수, 수제비, 양푼이비빔밥",5.00,2
4,굿후렌드식당,대전광역시 동구 성동로13번길 50,피자,"페페로니피자, 고구마피자, 포테이토피자, 콤비네이션피자",0.00,0
...,...,...,...,...,...,...
107,아마이카츠,"대전광역시 동구 동대전로 195-1, 1층",일식,"안심카츠, 등심카츠, 생선카츠, 치즈카츠, 카레, 돈카츠카레, 치킨가라아게카레",5.00,3
108,1989역전꼬마김밥 자양점,"대전광역시 동구 동대전로 221, 1층",분식,"김밥, 참치김밥, 라면, 쫄면, 우동, 우동, 등심돈까스, 치즈돈까스, 김치볶음밥, 떡볶이, 치즈떡볶이, 로제떡볶이, 김말이, 오징어튀김, 새우튀김, 치즈스틱, 만두",5.00,3
109,로충칭마라탕(우송대점),대전광역시 동구 동대전로 174-7,중식,"사브샤브, 마라탕, 마라샹궈,",4.45,260
110,한양,"대전광역시 동구 동대전로 178, 1층",한식,"스지 전골, 스지 볶음, 제육 볶음",5.00,1


In [77]:
menus_df = menus[['업소명','대분류', '메뉴', '별점', '별점 수']]


In [78]:
pd.set_option('max_colwidth', 100)
menus_df[['대분류','메뉴']][:1]

,대분류,메뉴
0,중식,"짜장면, 짬뽕, 우동, 라면, 냉면, 울면, 만두국, 콩국수, 볶음밥, 김치볶음밥, 짬뽕밥, 잡채밥, 오므라이스 오징어덮밥, 육개장, 뚝배기불고기, 갈비탕"


In [79]:
from sklearn.feature_extraction.text import CountVectorizer

menus_df['메뉴'] = menus_df['메뉴'].apply(lambda x : ('').join(x))

menus_df['메뉴']

# CountVectorizer
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(menus_df['메뉴'])

print(genre_mat.shape)

(112, 886)


In [80]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.13483997, 0.        ,
       0.08444007, 0.        , 0.04494666, 0.15745916, 0.03481553,
       0.        , 0.        , 0.06579517, 0.04222003, 0.        ,
       0.        , 0.        , 0.07987231, 0.        , 0.10497278,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06579517, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.03993615,
       0.        , 0.        , 0.        , 0.15632635, 0.        ,
       0.        , 0.        , 0.        , 0.04828045, 0.04828045,
       0.04494666, 0.        , 0.        , 0.13159034, 0.06579517,
       0.        , 0.        , 0.15632635, 0.        , 0.        ,
       0.08444007, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.05248

In [81]:
def find_sim_movie(df, sim_matrix, title_name, top_n=10):
    
    # 입력한 영화의 index
    title_movie = df[df['업소명'] == title_name]
    title_index = title_movie.index.values
    
    # 입력한 영화의 유사도 데이터 프레임 추가
    df["유사도"] = sim_matrix[title_index, :].reshape(-1,1)
    
    # 유사도 내림차순 정렬 후 상위 index 추출
    temp = df.sort_values(by="유사도", ascending=False)
    final_index = temp.index.values[ : top_n]
    
    return df.iloc[final_index]

In [82]:
similar_movies = find_sim_movie(menus_df, genre_sim, '써브웨이 우송대점', 10)
similar_movies[['업소명', '별점', "유사도"]]

,업소명,별점,유사도
111,써브웨이 우송대점,4.45,1.000000
70,솔파인 레스토랑,5.00,0.047619
60,이백장 돈가스 자양점,4.68,0.029788
58,복짬뽕,4.40,0.000000
72,쌍둥이네 떡볶이,4.35,0.000000
82,깻잎두마리치킨&푸드,4.38,0.000000
81,청년다방 대전우송대점,4.51,0.000000
80,크앙분식,4.32,0.000000
79,동궁찜닭,4.58,0.000000
78,만년닭강정,4.44,0.000000


In [83]:
percentile = 0.6
m = menus_df['별점 수'].quantile(percentile)
C = menus_df['별점'].mean()

def weighted_vote_average(record):
    v = record['별점 수']
    R = record['별점']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

menus_df['가중 별점'] = menus_df.apply(weighted_vote_average, axis=1)

In [84]:
temp = menus_df[['업소명','별점','별점 수','가중 별점']]
temp.sort_values('가중 별점', ascending=False)[:10]

,업소명,별점,별점 수,가중 별점
90,육형제,4.80,478,4.730783
53,아키타,4.67,414,4.622752
77,감동국수,4.76,140,4.621448
21,양푼이보리밥,4.90,68,4.619089
83,하오츠 마라탕 자양점,4.73,162,4.615139
106,더(THE)맛있는김밥,4.71,128,4.589605
57,밀플랜비 우송대점,4.63,310,4.582969
18,유가식당,4.67,134,4.571713
52,수제버거비앤디(B&D),4.71,92,4.569650
37,알촌,4.64,173,4.567163


In [85]:
def find_sim_movie(df, sim_matrix, title_name, top_n=10):
    
    # 입력한 영화의 index
    title_movie = df[df['업소명'] == title_name]
    title_index = title_movie.index.values
    
    # 입력한 영화의 유사도 데이터 프레임 추가
    df["유사도"] = sim_matrix[title_index, :].reshape(-1,1)
        
    # 유사도와 가중 평점순으로 높은 상위 index 추출 (자기 자신 제거)
    temp = df.sort_values(by=["유사도", "가중 별점"], ascending=False)
    temp = temp[temp.index.values != title_index]
    
    final_index = temp.index.values[:top_n]
    
    return df.iloc[final_index]

In [86]:
similar_movies = find_sim_movie(menus_df, genre_sim, '알촌', 10)
similar_movies[['업소명', '별점', "별점 수", "유사도"]]

,업소명,별점,별점 수,유사도
25,돈하우스,4.46,103,0.235441
41,문희네식당,4.80,4,0.170400
6,페리카나치킨,4.26,24,0.094176
49,맘스터치 우송대점,4.43,261,0.090167
46,굽네치킨 대전우송대점,4.51,261,0.080648
23,다송치킨호프,4.41,96,0.071657
51,차이하나(chayhana),4.68,44,0.059028
40,BHC우송대보건대점,4.36,577,0.059028
100,만리향,4.39,179,0.052058
8,만남의광장식당,4.69,18,0.040324
